In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error



USE_MIXED = False
USE_PRESSURE_TEMP = False
noise_std_dev = 1e-3


if USE_MIXED:
    filenames_vis = ['fog_data_vis_clear.dat', 'fog_data_vis_ice.dat', 'fog_data_vis_mixed.dat', 'fog_data_vis_water.dat']
    filenames_ir = ['fog_data_ir_clear.dat', 'fog_data_ir_ice.dat', 'fog_data_ir_mixed.dat', 'fog_data_ir_water.dat']
else:
    filenames_vis = ['fog_data_vis_clear.dat', 'fog_data_vis_ice.dat', 'fog_data_vis_water.dat']
    filenames_ir = ['fog_data_ir_clear.dat', 'fog_data_ir_ice.dat', 'fog_data_ir_water.dat']

target_i = 30 # Visibility
if USE_PRESSURE_TEMP:
    features_i = np.concatenate((np.arange(4, 17), np.arange(20, 30), np.arange(31,36))) # 13 ir bands, 10 vis bands, 3 angles, pressure, temp
else:
    features_i = np.concatenate((np.arange(4, 17), np.arange(20, 30), np.arange(31,34)))



data_ir_vis = []

for file_ir, file_vis in zip(filenames_ir, filenames_vis):
  raw_data_ir = pd.read_csv('fog_dataset/' + file_ir).to_numpy()
  raw_data_vis = pd.read_csv('fog_dataset/' + file_vis).to_numpy()

  data_ir = np.array([row[0].split() for row in raw_data_ir[25:]])
  data_vis = np.array([row[0].split()[1:] for row in raw_data_vis[41:]])

  if len(data_ir_vis) == 0:
    data_ir_vis = np.hstack([data_ir, data_vis])
  else:
    data_ir_vis = np.vstack([data_ir_vis, np.hstack([data_ir, data_vis])])


# Remove surface description to convert to float
data_ir_vis = data_ir_vis[:,:-1] 
data_ir_vis = data_ir_vis.astype(np.double)

nan_i = np.where(np.isnan(data_ir_vis))[0]
data_ir_vis = np.delete(data_ir_vis, nan_i, axis=0)



X = np.zeros((data_ir_vis.shape[0], len(features_i)))
y = np.zeros(data_ir_vis.shape[0])

for i in range(len(data_ir_vis)):
  X[i] = data_ir_vis[i, features_i]
  y[i] = data_ir_vis[i,target_i]

# 19 Data points contains nan. Not sure why. Band 32 culprit, 8th feature
nan_i = np.where(np.isnan(X))[0]
print(f'nani: {nan_i}')
X = np.delete(X, nan_i, axis=0)
y = np.delete(y, nan_i, axis=0)


# Scaling features
feature_scaler = StandardScaler()
X_scaled = feature_scaler.fit_transform(X)

X_scaled_noisy = X_scaled + np.random.normal(0, noise_std_dev, X_scaled.shape)

# Scaling target
target_scaler = StandardScaler()
y_scaled = target_scaler.fit_transform(y.reshape(-1, 1))


# First split: Separate out a test set (5% of the original dataset)
X_temp, X_test, y_temp, y_test = train_test_split(X_scaled_noisy, y_scaled, test_size=(1/20), random_state=16)

# Second split: Split the remaining data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=(1/19), random_state=16)

nani: []


In [22]:
 
print(X_scaled_noisy.shape)



cat_cols = []

num_cols = ['band20', 'band21', 'band22', 'band23', 'band24', 'band25',
       'band27', 'band28', 'band29', 'band30', 'band31', 'band32', 
       'band33', 'band11', 'band12', 'band13', 'band14', 'band15',
       'band16', 'band17', 'band18', 'band19', 'band26', 'sat_angle',
       'sun_angle', 'azimuth_angle']#, 'temperature', 'pressure']
target=["visibility"]

y_pd = pd.Series(y_scaled.flatten(), name='visibility')
X_pd = pd.DataFrame(X_scaled_noisy, columns=num_cols)
data_pd = X_pd.join(y_pd)
print(list(data_pd.columns))
#print(list(data_pd["visibility"]))

(149989, 26)
['band20', 'band21', 'band22', 'band23', 'band24', 'band25', 'band27', 'band28', 'band29', 'band30', 'band31', 'band32', 'band33', 'band11', 'band12', 'band13', 'band14', 'band15', 'band16', 'band17', 'band18', 'band19', 'band26', 'sat_angle', 'sun_angle', 'azimuth_angle', 'visibility']


In [21]:
train, test = train_test_split(data_pd, test_size=0.05)

In [31]:
from torch.utils.data import Dataset 
import copy

class TabularDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]
        
    def copy(self):
        return copy.deepcopy(self)
    def loc(self, idx):
        return self.X[idx], self.y[idx]
    
train_dataset = TabularDataset(X_train, y_train)
val_dataset = TabularDataset(X_val, y_val)
test_dataset = TabularDataset(X_test, y_test)

print(type(y_train))
print(type(y_val))
print(type(y_test))

from torch.utils.data import DataLoader 

train_dl = DataLoader(train_dataset, batch_size=256, shuffle=True)
val_dl = DataLoader(val_dataset, batch_size=256)
test_dl = DataLoader(test_dataset, batch_size=256) 

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [24]:
from pytorch_tabular import TabularModel
from pytorch_tabular.models import (
    CategoryEmbeddingModelConfig, 
    FTTransformerConfig, 
    TabNetModelConfig, 
    GatedAdditiveTreeEnsembleConfig, 
    TabTransformerConfig, 
    AutoIntConfig
)
from pytorch_tabular.config import DataConfig, OptimizerConfig, TrainerConfig, ExperimentConfig
from pytorch_tabular.models.common.heads import LinearHeadConfig

In [26]:
data_config = DataConfig(
    target=target, #target should always be a list.
    continuous_cols=num_cols,
)

trainer_config = TrainerConfig(
#     auto_lr_find=True, # Runs the LRFinder to automatically derive a learning rate
    batch_size=256,
    max_epochs=500,
    early_stopping="valid_loss", # Monitor valid_loss for early stopping
    early_stopping_mode = "min", # Set the mode as min because for val_loss, lower is better
    early_stopping_patience=5, # No. of epochs of degradation training will wait before terminating
    checkpoints="valid_loss", # Save best checkpoint monitoring val_loss
    load_best=True, # After training, load the best checkpoint
)

optimizer_config = OptimizerConfig()

head_config = LinearHeadConfig(
    layers="", # No additional layer in head, just a mapping layer to output_dim
    dropout=0.1,
    initialization="kaiming"
).__dict__ # Convert to dict to pass to the model config (OmegaConf doesn't accept objects)

In [30]:
from torch.utils.data import DataLoader

model_config = FTTransformerConfig(
    task="regression",
    learning_rate = 1e-3,
    head = "LinearHead", #Linear Head
    head_config = head_config, # Linear Head Config
)

tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
)

train_loader = DataLoader(train, batch_size=256, shuffle=True, num_workers=19)
val_loader = DataLoader(test, batch_size=256, shuffle=False, num_workers=19) 

#tabular_model.fit(train)
#tabular_model.evaluate(test)
tabular_model.fit(train_loader) 
tabular_model.evaluate(val_loader)

2024-03-04 14:56:44,034 - {pytorch_tabular.tabular_model:140} - INFO - Experiment Tracking is turned off

Seed set to 42


2024-03-04 14:56:44,055 - {pytorch_tabular.tabular_model:524} - INFO - Preparing the DataLoaders

2024-03-04 14:56:44,073 - {pytorch_tabular.tabular_datamodule:499} - INFO - Setting up the datamodule for          
regression task

2024-03-04 14:56:44,162 - {pytorch_tabular.tabular_model:574} - INFO - Preparing the Model: FTTransformerModel

...

┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name             ┃ Type                  ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ _backbone        │ FTTransformerBackbone │  271 K │
│ 1 │ _embedding_layer │ Embedding2dLayer      │  1.7 K │
│ 2 │ _head            │ LinearHead            │     33 │
│ 3 │ loss             │ MSELoss               │      0 │
└───┴──────────────────┴───────────────────────┴────────┘

Trainable params: 273 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 273 K                                                                                                
Total estimated model params size (MB): 1

Output()

/home/edgelab/.local/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 
'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the 
`num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.

/home/edgelab/.local/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 
'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the 
`num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.